In [1]:
# I will implement a basic ResNet model for image classification.
# The model will consist of several convolutional layers, batch normalization, and ReLU activations.
# The architecture will include skip connections to allow gradients to flow more easily through the network.
# The model will be built using PyTorch.
# The dataset used will have 10 classes, and the input images will need to be resized to 200x200 pixels.
# The model will be trained using the Adam optimizer and cross-entropy loss.
# data augmentation will be applied later. 

# Setup

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import time

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


# DataLoader

In [ ]:
# DataLoader

class ELImageDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.label = label  # integer label for the class
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # ensure grayscale

        if self.transform:
            image = self.transform(image)

        return image, self.label

def get_all_class_loaders(root_dir, image_size=(224, 224), batch_size=32, num_workers=2):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    all_loaders = {}

    # Auto-assign label IDs
    class_folders = sorted(os.listdir(root_dir))
    class_to_label = {name: idx for idx, name in enumerate(class_folders)}

    for class_name in class_folders:
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        label = class_to_label[class_name]
        train_dir = os.path.join(class_path, 'train')
        test_dir = os.path.join(class_path, 'test')

        train_dataset = ELImageDataset(train_dir, label, transform=transform)
        test_dataset = ELImageDataset(test_dir, label, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        all_loaders[class_name] = {
            'train': train_loader,
            'test': test_loader
        }

    return all_loaders

root_dir = 'D:\jungha\2025 Spring\MEC510\term project\Processed_Data\images_crop'
loaders = get_all_class_loaders(root_dir)

In [16]:
# resnet model

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
    

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [17]:
# initialize model 
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10)  # Adjust the number of classes as needed
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
# print(model)
# print(model.conv1.weight.shape)  # Check the shape of the first conv layer weights

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 